## Importing libraries and installing xlrd for reading xlsx files

In [1]:
import numpy as np
import pandas as pd
import os

## Reading xlsx files and joining them together
#### Please download all excel files and put in same directory as your current notebook

In [2]:
path = os.getcwd()
filename = '\\body_composition_measurements_subject_'
subjects = ['A', 'B', 'C', 'D', 'E']

df = pd.DataFrame()
for subject in subjects:
    dftemp = pd.read_excel('{}{}{}.xlsx'.format(path, filename, subject))
    df = df.append(dftemp, ignore_index=True)

df.shape

(186, 11)

In [3]:
df.head()

,Subject,Date (yy-mm-dd),Height (cm),Bodyweight (kg),BMI (score),BMI classification (-/0/+/++),Bodyfat (%),Skeletal muscle (%),Resting metabolism (kcal),Visceral fat (level),Visceral fat classification (-/0/+/++)
0,A,2020-10-17,180,78.6,24.3,0,25.3,36.6,1750,7.0,0
1,A,2020-10-18,180,78.0,24.1,0,25.5,36.3,1738,6.0,0
2,A,2020-10-19,180,77.8,24.0,0,25.6,36.2,1734,6.0,0
3,A,2020-10-20,180,77.8,24.0,0,26.0,35.9,1732,7.0,0
4,A,2020-10-21,180,79.2,24.4,0,26.5,35.7,1753,7.0,0


In [4]:
df.columns

Index(['Subject', 'Date (yy-mm-dd)', 'Height (cm)', 'Bodyweight (kg)',
       'BMI (score)', 'BMI classification (-/0/+/++)', 'Bodyfat (%)',
       'Skeletal muscle (%)', 'Resting metabolism (kcal)',
       'Visceral fat (level)', 'Visceral fat classification (-/0/+/++)'],
      dtype='object')

In [16]:
import panel as pn
import hvplot.pandas
from panel.interact import interact

pn.extension()

subs = ['A', 'B', 'C', 'D', 'E']
var = pn.widgets.ToggleGroup(options=subs, behavior='radio')

@pn.depends(var)
def tabs(sub):
    dft = df[df['Subject'] == sub]
    return pn.Tabs(
        ('Weight' , dft['Bodyweight (kg)'].hvplot(x= 'Days')),
        ('BMI', dft['BMI (score)'].hvplot(x= 'Days')),
        ('Bodyfat', dft['Bodyfat (%)'].hvplot(x= 'Days')),
        ('Muscle', dft['Skeletal muscle (%)'].hvplot(x= 'Days')))

interact(tabs, sub=var)

Column
    [0] Column
        [0] RadioButtonGroup(options=['A', 'B', 'C', 'D', 'E'], value='A')
    [1] Row
        [0] Tabs
            [0] HoloViews(Curve, name='Weight')
            [1] HoloViews(Curve, name='BMI')
            [2] HoloViews(Curve, name='Bodyfat')
            [3] HoloViews(Curve, name='Muscle')

I know x value are not shown correctly, i will fix it. i just learned to work with widgets. 